In [56]:
#### Imports ####
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib.pyplot import imshow, show, subplot,title,axis
from matplotlib.patches import Circle
from skimage.io import imread
import skimage.util
import skimage as sk
from timeit import timeit
from skimage.feature import corner_peaks,peak_local_max
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap
import os
from skimage import exposure, filters, morphology
from cellpose import models, io, denoise, utils
from cellpose.io import imread, logger_setup
import pandas as pd
from skimage.measure import regionprops, label, regionprops_table
from skimage.feature import blob_log, graycomatrix, graycoprops
from skimage.exposure import rescale_intensity
from skimage import transform
from skimage.draw import polygon_perimeter
from scipy.ndimage import binary_erosion


from Functions_George import compute_cdf, C, custom_colormap, detect_outline, load_images_from_folder

In [57]:
# import os

# os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"  
# os.environ["OMP_NUM_THREADS"] = "1"  

paths = [r"C:\Users\George\Desktop\Final_Project\Images\Control", r"C:\Users\George\Desktop\Final_Project\Images\Penetramax"]

In [58]:
io.logger_setup()

all_props = []
Nuclei = []

for path in paths:
    Images = load_images_from_folder(path)

    #Model without and with denoising
    # model = models.Cellpose(model_type='nuclei', gpu=True)
    model = denoise.CellposeDenoiseModel(gpu=True, model_type="nuclei", restore_type='denoise_nuclei')
    
    for i in range(len(Images)):
        #1.Isolating the red channel for each Image
        Red_channel = np.copy(Images[i][:, :, 0])

        #2 Apply Adaptive Histogram equalization (play with kernal size and clip_limit), works better here
        Red_channel = skimage.exposure.equalize_adapthist(Red_channel, kernel_size=10, clip_limit=0.01, nbins=512)

        #3.Segmentation
        masks, flows, styles, diams = model.eval(Red_channel, diameter=60, channels=[0, 0])
    
        # fig, axes = plt.subplots(1, 3, figsize=(20, 6))
        # ax = axes.ravel()

        # ax[0].imshow(Images[i])
        # ax[0].set_title('Original Image')
        # ax[1].imshow(Red_channel, cmap='gray')
        # ax[1].set_title('Brightened Image')
        # ax[2].imshow(Images[i])
        # ax[2].imshow(masks, alpha = 0.9*(masks>0.01), cmap = custom_colormap())
        # ax[2].set_title('Segmented Image')

        # for a in ax:
        #     a.axis('off')

        # plt.suptitle(f'Image {i + 1}', x = 0.5, y = 0.97, fontsize = 25)
        # plt.tight_layout()
        # plt.show()

        props = regionprops(masks, spacing=(1, 1))
        for j, prop in enumerate(props):

            min_row, min_col, max_row, max_col = prop['bbox']
            mask = prop.image
            New_Red = Images[i][min_row:max_row, min_col:max_col, 0] * mask
            Normalized_Red = rescale_intensity(New_Red, in_range='image', out_range=(0, 1))  
            Normalized_Red = filters.gaussian(Normalized_Red, sigma=1)
            # threshold = filters.threshold_otsu(Normalized_Red)
            threshold = 0.3
            New_binary_image = np.clip((Normalized_Red > threshold).astype(int) + detect_outline(mask), 0, 1)
            
            ###VISUALIZATION OF THE NUCLEI###
            # if path == paths[0]:
            #     fig, ax = plt.subplots(1, 3, figsize = (8,6))
    
            #     ax[0].imshow(mask, cmap = 'gray')
            #     ax[0].set_title('Mask', fontsize = 14)
            #     ax[1].imshow(Normalized_Red, cmap = 'gray')
            #     ax[1].set_title('Intensity image (Red channel)', fontsize = 14)
            #     ax[2].imshow(New_binary_image, cmap = 'gray')
            #     ax[2].set_title('Binary nucleus image', fontsize = 14)
            #     plt.tight_layout()
            #     plt.show()
                
            def Category(region):
                category_value = 0 if path == paths[0] else 1
                return category_value
            def Intensity_std(region, intensities):
                return np.std(intensities[region], ddof=1)
            # def Homogeneity(region):
            #     glcm = graycomatrix(region, distances=[1, 2], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4, np.pi, 5*np.pi/4, 3*np.pi/2, 7*np.pi/4], levels=100, symmetric=True, normed=True)
            #     homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
            #     return homogeneity

            table = regionprops_table(New_binary_image, Normalized_Red, properties = ['area', 'area_bbox', 'area_filled', 'area_convex','axis_major_length',
            'axis_minor_length', 'eccentricity', 'euler_number', 'equivalent_diameter_area', 'feret_diameter_max', 'intensity_max', 
            'mean_intensity','intensity_min', 'solidity', 'perimeter', 'perimeter_crofton', 'num_pixels','orientation'],  extra_properties=(Category, Intensity_std,))
            
            area_perc = 100 - (table['area_filled'] - table['area']) / table['area_filled']
            table['area_percentage'] = area_perc

            for k in range(len(table['euler_number'])):  # Iterate through each region in the table
                row = {key: table[key][k] for key in table}
                all_props.append(row)

            Nuclei.append(Normalized_Red)

df = pd.DataFrame(all_props)
df.to_csv('Segmented_Nuclei_train.csv', index=False)

creating new log file
2024-05-30 18:01:35,218 [INFO] WRITING LOG OUTPUT TO C:\Users\George\.cellpose\run.log
2024-05-30 18:01:35,220 [INFO] 
cellpose version: 	3.0.8 
platform:       	win32 
python version: 	3.11.9 
torch version:  	2.2.2+cu118
2024-05-30 18:01:35,484 [INFO] >> denoise_nuclei << model set to be used
2024-05-30 18:01:35,484 [INFO] ** TORCH CUDA version installed and working. **
2024-05-30 18:01:35,484 [INFO] >>>> using GPU
2024-05-30 18:01:35,672 [INFO] >>>> model diam_mean =  17.000 (ROIs rescaled to this size during training)
2024-05-30 18:01:35,672 [INFO] >> nuclei << model set to be used
2024-05-30 18:01:35,692 [INFO] ** TORCH CUDA version installed and working. **
2024-05-30 18:01:35,693 [INFO] >>>> using GPU
2024-05-30 18:01:35,762 [INFO] >>>> loading model C:\Users\George\.cellpose\models\nucleitorch_0
2024-05-30 18:01:36,613 [INFO] >>>> model diam_mean =  17.000 (ROIs rescaled to this size during training)
2024-05-30 18:02:19,530 [INFO] >> denoise_nuclei << mode

In [59]:
# N = 1290
# ira = np.zeros((Nuclei[N].shape[0], Nuclei[N].shape[1], 3))
# ira[:, :, 0], ira[:, :, 1], ira[:, :, 2] = Nuclei[N], 0, 0
# imshow(ira)
# ####Note: mean_intensity is based on the intensity image of regionprops####